In [1]:
import pandas as pd
from io import StringIO
import io
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
from sklearn.preprocessing import LabelEncoder
import ast
from tensorflow.keras import layers
import keras

In [2]:
df=pd.read_excel('legit_ones_5l_vectors.xlsx')

In [3]:
encoder=LabelEncoder()
df['label']=encoder.fit_transform(df['label'])

In [4]:
# max lenth of is 45 for idioms
r=[]
for i in range(df.shape[0]):
    x = ast.literal_eval(df['all_idiom_5l_each_word'][i])
    r.append(x)
    

In [5]:
for j in range(len(r)):
    for i in range(45-len(r[j])):
        r[j].append(0)

In [6]:
data1=pd.concat([pd.DataFrame(r),pd.DataFrame(list(df['label']),columns=['y'])],axis=1)
data1

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,y
0,0.706041,-2.400280,1.441445,-1.572476,3.178705,-0.989343,-2.965439,-1.656484,-0.818308,2.228827,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,-2.911975,0.466270,-0.744243,-1.081924,-2.690621,-1.982522,-1.727190,-2.760475,-1.732051,-0.339831,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,-1.264573,0.961804,3.210402,-0.762324,0.473270,-2.267075,-0.246786,2.582364,0.359696,0.347087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3.912891,0.240082,2.398815,0.254071,0.138524,1.709964,0.659743,0.193390,0.474542,2.163204,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.461462,-0.827581,-1.058371,-2.533165,2.605412,-2.262347,5.604730,-0.617172,0.278911,0.153157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24805,-1.573374,4.653001,1.094262,-2.634780,2.059686,0.240416,0.289104,-0.246852,-0.731543,0.361849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
24806,0.197703,2.343588,2.757625,-0.619617,1.132591,0.914919,0.469362,3.042929,-0.427493,3.312955,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
24807,-2.161072,2.054279,2.774971,1.189215,4.888242,-0.735887,1.086011,2.906706,0.553608,2.586283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
24808,4.936450,7.411347,1.970084,-2.220847,-1.292620,0.453957,4.809259,2.020978,-3.288856,-2.603212,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [7]:
train_idiom=data1[(data1['y']==0)&(data1.index<20000)].reset_index(drop=True)
train_idiom.shape

(16742, 46)

In [8]:
train_literal=data1[(data1['y']==1)&(data1.index<20000)].reset_index(drop=True)
train_literal_bias=pd.DataFrame()
train_literal_bias=pd.concat([train_literal_bias,train_literal,train_literal,train_literal,train_literal,train_literal,train_literal[0:452]],axis=0)
train_literal_bias.shape

(16742, 46)

In [9]:
train_literal[0:452]

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,y
0,0.200794,1.746465,-2.469869,-2.413901,3.175566,2.754219,3.479995,0.845903,-2.629498,6.356821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,-4.471947,0.251455,-0.852516,0.875013,0.907998,-5.074784,-1.981690,-1.358121,1.145342,2.465438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,-0.398004,-1.787525,0.093280,-1.910716,3.612620,-2.922696,-0.794799,-0.683657,2.075389,1.013283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,-3.163574,-2.277607,0.512381,-4.007491,-0.503701,-2.844902,-4.045784,-0.653298,-2.134147,0.967743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,-1.647244,1.636005,-0.747913,-1.711509,4.521041,0.742066,4.090480,-0.664268,-1.796380,5.209273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,-2.845213,3.246986,-0.339011,5.081264,2.669884,-4.743480,4.011475,-1.459684,6.556819,1.923460,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
448,-1.996757,0.283110,-1.624118,0.087100,0.498333,-0.903049,2.904166,-0.832911,4.955476,0.796886,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
449,-1.657352,2.667484,2.989386,-2.029863,2.197638,-0.436760,-0.626940,3.460732,-0.748058,4.563299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
450,-1.194216,2.536058,1.035588,0.031019,-0.470849,-2.807090,1.284870,-0.393757,0.377447,-0.401311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [10]:
16742-3258*5

452

In [11]:
train_idiom=data1[(data1['y']==0)&(data1.index<20000)].reset_index(drop=True)[0:3258]
train_literal=data1[(data1['y']==1)&(data1.index<20000)].reset_index(drop=True)[0:3258]
train=pd.concat([train_idiom,train_literal],axis=0).reset_index(drop=True)
train= train.sample(frac = 1).reset_index(drop=True)

test_literal=data1[(data1['y']==1)&(data1.index>20000)].reset_index(drop=True)
test_idiom=data1[(data1['y']==0)&(data1.index>20000)].reset_index(drop=True)[0:769]
test=pd.concat([test_idiom,test_literal],axis=0).reset_index(drop=True)
test=test.sample(frac = 1).reset_index(drop=True)


In [12]:
from sklearn.linear_model import LogisticRegression
x_train=train.iloc[:,:45]
y_train=train['y']
x_test=test.iloc[:,:45]
y_test=test['y']


x_train=np.array(x_train)
x_test=np.array(x_test)


In [29]:
X=data1.iloc[:,0:45]
y=data1.iloc[:,45:46]
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
x_train=np.array(x_train)
x_test=np.array(x_test)

In [30]:
x_train=np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))
x_test=np.reshape(x_test,(x_test.shape[0],1,x_test.shape[1]))

In [31]:
model = keras.Sequential()
# model.add(keras.Input(shape=(x_train.shape[1],1)))

model.add(keras.Input(shape=(1,x_train.shape[2])))

model.add(layers.Bidirectional(layers.LSTM(256, return_sequences=True, activation="relu")))

model.add(layers.LSTM(128, name="lstm_layer2"))

# model.add(LSTM(512, activation='exponential'))
model.add(layers.Dense(64, activation='exponential'))

model.add(layers.Dense(30,activation='hard_sigmoid'))

model.add(layers.Dense(10,activation='tanh'))

# model.add(layers.Dense(16,activation='hard_sigmoid'))

# model.add(LSTM(64, activation='exponential'))
# model.add(layers.Dense(10,activation='hard_sigmoid'))
# model.add(layers.Dense(10,activation='tanh'))

model.add(layers.Dense(1,activation='sigmoid'))

print(model.summary())
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=200, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 1, 512)            618496    
_________________________________________________________________
lstm_layer2 (LSTM)           (None, 128)               328192    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 30)                1950      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 957,215
Trainable params: 957,215
Non-trainable params: 0
________________________________________________

260/260 - 5s - loss: 0.0250 - accuracy: 0.9946
Epoch 119/200
260/260 - 5s - loss: 0.0247 - accuracy: 0.9952
Epoch 120/200
260/260 - 4s - loss: 0.0187 - accuracy: 0.9962
Epoch 121/200
260/260 - 4s - loss: 0.0227 - accuracy: 0.9954
Epoch 122/200
260/260 - 4s - loss: 0.0230 - accuracy: 0.9953
Epoch 123/200
260/260 - 4s - loss: 0.0235 - accuracy: 0.9950
Epoch 124/200
260/260 - 4s - loss: 0.0229 - accuracy: 0.9952
Epoch 125/200
260/260 - 4s - loss: 0.0218 - accuracy: 0.9955
Epoch 126/200
260/260 - 4s - loss: 0.0220 - accuracy: 0.9959
Epoch 127/200
260/260 - 4s - loss: 0.0257 - accuracy: 0.9949
Epoch 128/200
260/260 - 4s - loss: 0.0226 - accuracy: 0.9956
Epoch 129/200
260/260 - 4s - loss: 0.0240 - accuracy: 0.9954
Epoch 130/200
260/260 - 4s - loss: 0.0201 - accuracy: 0.9966
Epoch 131/200
260/260 - 4s - loss: 0.0251 - accuracy: 0.9955
Epoch 132/200
260/260 - 5s - loss: 0.0225 - accuracy: 0.9957
Epoch 133/200
260/260 - 5s - loss: 0.0229 - accuracy: 0.9956
Epoch 134/200
260/260 - 4s - loss: 0.0

[0.6560125350952148, 0.8814118504524231]

In [ ]:
# model = Sequential()
# # model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(1, x_train.shape[1])))
# model.add(layers.Bidirectional(layers.LSTM(500, return_sequences=True, activation="tanh",input_shape=(1, x_train.shape[2]))))
# model.add(LSTM(300, activation='tanh'))
# model.add(Dense(10, activation='sigmoid'))

# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(x_train,y_train, batch_size=32, epochs=10)

In [32]:
o=model.predict(x_test)

In [33]:
Y_prep=[]
for i in range(len(o)):
    Y_prep.append(round(o[i][0]))

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test,Y_prep)
print (confusion_matrix)

[[6441  405]
 [ 566  776]]


In [2]:
print("Idiomatic classification accuracy! : ",(6441 / (6441+405)))

Idiomatic classification accuracy! :  0.9408413672217353


In [3]:
print("Literal classification accuracy! : ",(776 / (776+566)))

Literal classification accuracy! :  0.5782414307004471


In [5]:
print("Overall classification accuracy! : ",((776+6441) / (6441+405+566+776)))

Overall classification accuracy! :  0.8814118221787982
